In [2]:
import numpy as np
import scipy as sc
from enum import Enum
from typing import Sequence, Union
from collections import OrderedDict
import functools 

np.set_printoptions(suppress=True, precision=3) # to avoid 1e-17 expressions in rotation matrix

def rotation_matrix_x(angle: float) -> np.ndarray:
    """Generate a basic 4x4 rotation matrix about the X axis."""
    s = np.sin(angle)
    c = np.cos(angle)

    matrix = np.array([1, 0, 0, 0, 0, c, -s, 0, 0, s, c, 0, 0, 0, 0, 1]).reshape((4, 4))

    return matrix


def rotation_matrix_y(angle: float) -> np.ndarray:
    """Generate a basic 4x4 rotation matrix about the Y axis."""
    s = np.sin(angle)
    c = np.cos(angle)

    matrix = np.array([c, 0, s, 0, 0, 1, 0, 0, -s, 0, c, 0, 0, 0, 0, 1]).reshape((4, 4))

    return matrix


def rotation_matrix_z(angle: float) -> np.ndarray:
    """Generate a basic 4x4 rotation matrix about the Z axis."""
    s = np.sin(angle)
    c = np.cos(angle)

    matrix = np.array([c, -s, 0, 0, s, c, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]).reshape((4, 4))

    return matrix


def translation_matrix(xyz: Sequence[float]) -> np.ndarray:
    """Generate a basic 4x4 translation matrix."""
    # validate
    if len(xyz) != 3:
        raise Exception('len(xyz) must be 3')

    matrix = np.eye(4)
    matrix[:-1, -1] = xyz

    return matrix

def DH(conf):
    '''
    Calculates the Denavit-Hartenberg Matrix
        d: offset along previous z to the common normal
        θ: angle about previous z, from old x to new x
        r: length of the common normal, if it is a revolute joint this is the radius about previous z
        α: angle about common normal, from old z axis to new z axis
    '''
    if len(conf) != 4:
        raise Exception('Need exactly 4 Denavit-Hartenberg parameters, provided %i.' % len(conf))
        
    d, Theta, r, alpha = conf
    
    T = np.eye(4, dtype=np.float32)

    cTheta = np.cos(Theta/180.0*np.pi)
    sTheta = np.sin(Theta/180.0*np.pi)
    calpha = np.cos(alpha/180.0*np.pi)
    salpha = np.sin(alpha/180.0*np.pi)
    
    T[np.ix_([0],[0])] = cTheta
    T[np.ix_([0],[1])] = -sTheta * calpha
    T[np.ix_([0],[2])] = sTheta * salpha
    T[np.ix_([0],[3])] = r * cTheta
    
    T[np.ix_([1],[0])] = sTheta
    T[np.ix_([1],[1])] = cTheta * calpha
    T[np.ix_([1],[2])] = -cTheta * salpha
    T[np.ix_([1],[3])] = r * sTheta

    T[np.ix_([2],[1])] = salpha
    T[np.ix_([2],[2])] = calpha
    T[np.ix_([2],[3])] = d

    return T


print("Computing for MELFA RV-E2")

configuration_RV_E2 = OrderedDict([
                ('T1', [270.0, -90.0, 100.0, 90]),
                ('T2', [0.0, -90.0, 250.0, 0.0]),
                ('T3', [0.0, 0.0, 130.0, 90.0]),
                ('T4', [250.0, 0.0, 0.0, -90.0]),
                ('T5', [0.0, 0.0, 85.0, -90.0])
                ])

T_RV_E2 = OrderedDict()
for name, conf in configuration_RV_E2.items():
    print('Computed %s = \n' % name, DH(conf))
    T_RV_E2[name] = DH(conf)

print("T6 (endpoint) = ")
T6_RV_E2 = functools.reduce(np.dot, T_RV_E2.values())
print(T6_RV_E2)


print("\n\nComputing for MELFA RV-E2")

configuration_RV_E3J = OrderedDict([
                ('T1', [270.0, -90.0, 100.0, -90]),
                ('T2', [0.0, -90.0, 250.0, 0.0]),
                ('T3', [0.0, 90.0, 130.0, 0.0]),
                ('T4', [250.0, -90.0, 0.0, -90.0])
                ])

T_RV_E3J = OrderedDict()
for name, conf in configuration_RV_E3J.items():
    print('Computed %s = \n' % name, DH(conf))
    T_RV_E3J[name] = DH(conf)

print("T5 (endpoint) = ")
T5_RV_E3J = functools.reduce(np.dot, T_RV_E3J.values())
print(T5_RV_E3J)



Computing for MELFA RV-E2
Computed T1 = 
 [[   0.    0.   -1.    0.]
 [  -1.    0.   -0. -100.]
 [   0.    1.    0.  270.]
 [   0.    0.    0.    1.]]
Computed T2 = 
 [[   0.    1.   -0.    0.]
 [  -1.    0.   -0. -250.]
 [   0.    0.    1.    0.]
 [   0.    0.    0.    1.]]
Computed T3 = 
 [[  1.  -0.   0. 130.]
 [  0.   0.  -1.   0.]
 [  0.   1.   0.   0.]
 [  0.   0.   0.   1.]]
Computed T4 = 
 [[  1.  -0.  -0.   0.]
 [  0.   0.   1.   0.]
 [  0.  -1.   0. 250.]
 [  0.   0.   0.   1.]]
Computed T5 = 
 [[ 1. -0. -0. 85.]
 [ 0.  0.  1.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]
T6 (endpoint) = 
[[  -0.    1.    0.   -0.]
 [  -0.    0.   -1.  150.]
 [  -1.   -0.    0. -195.]
 [   0.    0.    0.    1.]]


Computing for MELFA RV-E2
Computed T1 = 
 [[   0.    0.    1.    0.]
 [  -1.    0.    0. -100.]
 [   0.   -1.    0.  270.]
 [   0.    0.    0.    1.]]
Computed T2 = 
 [[   0.    1.   -0.    0.]
 [  -1.    0.   -0. -250.]
 [   0.    0.    1.    0.]
 [   0.    0.    0.    1.]]
Computed 